## 安装依赖

如果当前环境已经有相关依赖了，则不用执行

In [ ]:
!pip install transformers[torch] datasets==3.6.0 evaluate seqeval

## 加载数据

In [ ]:
from datasets import load_dataset

datasets = load_dataset("conll2003")

In [ ]:
# 你可以查看dataset对象都封装了什么东西
datasets

## 数据集可视化

为了能够进一步理解数据长什么样子，下面的函数将从数据集里随机选择几个例子进行展示；

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

## 数据预处理

transformer预训练模型在预训练的时候通常使用的是subword(子词)；

如果我们的文本输入已经被切分成了word，那么这些word还会被我们的tokenizer继续切分；就是给字词拼接上特殊Token，如（[CLS]）；根据时态语态再次划分等；

例如：

In [ ]:
from transformers import AutoTokenizer

task = "ner" #需要是"ner", "pos" 或者 "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_input = tokenizer(datasets["train"][4]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

`is_split_into_words=True` 的作用是告诉 tokenizer，不要把它当作一整句字符串去重新用空格切词，因为他已经是分割好的子词列表了，而是逐个 token 处理，然后再做 subword 拆分；

由于tokenizer将数据集里面的子词进一步分词后，tags的长度和subword的长度就对不上了；所以需要进行数据对齐；

In [ ]:
len(datasets["train"][4][f"{task}_tags"]), len(tokenized_input["input_ids"])

## 数据对齐

对齐label的两种方式：
- 多个subtokens对齐一个word，对齐一个label
- 多个subtokens的第一个subtoken对齐word，对齐一个label，其他subtokens直接赋予-100；

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else datasets["train"][4][f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

word_ids 这个API会返回每个subwords对应的wordId，因为一个词也会被拆成多个词；

只需要将wordId对不上的元素，自定义处理一下，比如换成-100；像一下特殊的Token[CLS][SEP]等就是对不上的；就替换成-100，

这样再计算loss时，可以自动忽略掉这些特殊Token；

## 封装数据预处理函数

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # 把特殊token的id设置成-100，在训练阶段计算损失值时会自动忽略
            if word_idx is None:
                label_ids.append(-100)
            # 我们为每个单词的第一个token设置标签。
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])

            # 对于单词中的其他token，我们将标签设置为当前标签或 -100，具体取决于label_all_tokens的配置。
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenize_and_align_labels(datasets['train'][:5])

## 数据映射

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

## 加载预训练模型

In [ ]:
from transformers import AutoModelForTokenClassification

label_list = datasets["train"].features[f"{task}_tags"].feature.names
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

## 创建训练参数配置器

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    f"test-{task}",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to="none"
)

## 获取数据加载器

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

## 获取评估指标

In [ ]:
import evaluate

metric = evaluate.load("seqeval")

## 封装评估函数

In [ ]:
# 融合多个指标的评估函数
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

## 构建训练器

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## 开始训练

In [ ]:
trainer.train()

## 模型评估

In [ ]:
trainer.evaluate()

## 模型预测

为了在完成训练后计算出每个类别的准确率/召回率/f1，我们可以对 predict 方法的结果应用与之前相同的函数：

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results